In [27]:
import os

In [28]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

In [29]:
from dotenv import load_dotenv
load_dotenv()

True

In [30]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
model = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0.7,convert_system_message_to_human=True)